In [ ]:


# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import gc
pd.set_option('display.max_columns', 500)

def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns


def bb__agg(num_rows = None, nan_as_category = True):
    bb = pd.read_csv('../input/home-credit-default-risk/bureau_balance.csv', nrows = num_rows)
    
    # DPD (Days Past Due) 'ye düşmüşmü, düşmemişmi?'
    liste = ['1','2','3','4','5']
    bb['NEW_FLAG'] = bb['STATUS'].apply(lambda x : 1 if (x in liste) else ("X" if x == "X" else 0))
    
    bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    bb.drop("NEW_FLAG_X", inplace=True, axis = 1)
    bb_cat.remove('NEW_FLAG_X')
    
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    
    del bb
    gc.collect()
    
    return  bb_agg, bb_cat


def bureau_(num_rows = None, nan_as_category = True):
    bu = pd.read_csv('../input/home-credit-default-risk/bureau.csv', nrows = num_rows)
    
    # Kredi Aktive ve Closed toplam Sayılarını ve Oranlarını hesaplamak
    temp_bu = bu[['SK_ID_CURR', 'CREDIT_ACTIVE']]
    temp_bu = pd.get_dummies(temp_bu)
    temp_bu = temp_bu.groupby('SK_ID_CURR').agg({'CREDIT_ACTIVE_Active':'sum','CREDIT_ACTIVE_Closed':'sum' })
    temp_bu.columns = ['CREDIT_ACTIVE_Active_Count','CREDIT_ACTIVE_Closed_Count']
    temp_bu['CREDIT_ACTIVE_Active_ratio'] = temp_bu['CREDIT_ACTIVE_Active_Count'] / (temp_bu['CREDIT_ACTIVE_Active_Count'] + temp_bu['CREDIT_ACTIVE_Closed_Count'])
    temp_bu['CREDIT_ACTIVE_Closed_ratio'] = temp_bu['CREDIT_ACTIVE_Closed_Count'] / (temp_bu['CREDIT_ACTIVE_Active_Count'] + temp_bu['CREDIT_ACTIVE_Closed_Count'])
    bu = bu.merge(temp_bu, on=['SK_ID_CURR'], how='left')
    
    # Kredi DAYS_CREDIT'i SK_ID_CURR bazında sıralayarak NEW_DAYS_DIFF değişkeni üretmek kredi alma frekansı bilgisi verebilir.
    
    temp_bu = bu[['SK_ID_CURR', 'SK_ID_BUREAU', 'DAYS_CREDIT']].groupby(by = ['SK_ID_CURR']).apply(lambda x: x.sort_values(['DAYS_CREDIT'], ascending = True)).reset_index(drop = True)
    temp_bu['NEW_DAYS_DIFF'] = temp_bu.groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT'].diff()
    temp_bu = temp_bu[['SK_ID_BUREAU','NEW_DAYS_DIFF']]
    temp_bu['NEW_DAYS_DIFF'] = temp_bu['NEW_DAYS_DIFF'].fillna(0)
    bu = bu.merge(temp_bu, on=['SK_ID_BUREAU'], how='left')
    
    # Active ve Closed Krediler için kredi erken kapanmışmı? 
    bu.loc[(bu['CREDIT_ACTIVE'] == 'Active') & (bu['DAYS_CREDIT_ENDDATE'] < 0), 'NEW_EARLY_ACTİVE'] = 1
    bu.loc[(bu['CREDIT_ACTIVE'] == 'Closed') & (abs(bu['DAYS_CREDIT_ENDDATE']) < abs(bu['DAYS_ENDDATE_FACT']) ), 'NEW_EARLY_CLOSED'] = 1
    
    # Uzatılmış Kredilerin 1 ile değiştirilmesi
    prolong = [1,2,3,4,5,6,7,8,9]
    bu['CNT_CREDIT_PROLONG'].replace(prolong, 1, inplace= True)
    
    
    # Kişi Kaç farklı kredi tipi almış
    temp_bu = bu[['SK_ID_CURR', 'CREDIT_TYPE']].groupby(by=['SK_ID_CURR'])['CREDIT_TYPE'].nunique().reset_index().rename(index=str, columns={'CREDIT_TYPE': 'NEW_BUREAU_LOAN_TYPES'})
    bu = bu.merge(temp_bu, on=['SK_ID_CURR'], how='left')
    
    # Borç Oranı
    bu['NEW_DEPT_RATİO'] = bu['AMT_CREDIT_SUM_DEBT'] / (bu['AMT_CREDIT_SUM']+1)
    
    # Kredi Tiplerinin 'others' ile değiştirilmesi
    credit_type = ['Loan for working capital replenishment',
       'Loan for business development', 'Real estate loan',
       'Unknown type of loan', 'Another type of loan',
       'Cash loan (non-earmarked)', 'Loan for the purchase of equipment',
       'Mobile operator loan', 'Interbank credit',
       'Loan for purchase of shares (margin lending)']
       
    bu['CREDIT_TYPE'].replace(credit_type, 'others', inplace= True)
    
    # Aylık Ödeme Oranı
    bu['NEW_AMT_ANNUITY_RATİO'] = bu['AMT_ANNUITY'] / bu['AMT_CREDIT_SUM']
    
    # Kredi güncellenmesi yenimi ?
    bu['NEWS_DAYS_CREDIT_UPDATE'] = bu['DAYS_CREDIT_UPDATE'].apply(lambda x : 'old' if x < -90 else 'new')
    
    # 'CREDIT_CURRENCY' değişkenini düşürmek 
    bu.drop('CREDIT_CURRENCY',inplace=True,axis = 1)
    
    del temp_bu
    gc.collect()
    
    return bu


def combine(bureau,bb_agg):
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)
    return bureau


# Preprocess bureau.csv and bureau_balance.csv
def bureau_and_balance(num_rows = None, nan_as_category = True):
   
    bb_agg, bb_cat = bb__agg(num_rows,nan_as_category)
    bureau = bureau_(num_rows,nan_as_category)
    bureau, bureau_cat = one_hot_encoder(bureau,nan_as_category )
    bureau = combine(bureau,bb_agg)
    
    # Bureau and bureau_balance numeric features
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum'],
        "CREDIT_ACTIVE_Active_Count":["mean"],
        "CREDIT_ACTIVE_Closed_Count":["mean"],
        "CREDIT_ACTIVE_Active_ratio":["mean"],
        "NEW_DAYS_DIFF":['max', 'mean'],
        "NEW_EARLY_ACTİVE":['mean'],
        "NEW_EARLY_CLOSED":['mean'],
        "NEW_BUREAU_LOAN_TYPES":['mean'],
        "NEW_DEPT_RATİO":['max', 'mean'],
        "NEW_AMT_ANNUITY_RATİO":['max', 'mean']
        }
    
    for col in bb_cat:
        num_aggregations[col + "_MEAN"] = ['mean']
    

    
    
    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']
    
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')
    del active, active_agg
    gc.collect()
    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
    del closed, closed_agg, bureau,bb_agg
    gc.collect()
    return bureau_agg